In [1]:
import pandas as pd
import numpy as np
from langdetect import detect
import re
pd.options.mode.chained_assignment = None

In [2]:
pd.set_option('display.max_columns', 100) # Para mostrar todas las columnas
# pd.set_option('display.max_rows', 100) # Para mostrar todas las filas

In [3]:
data_raw = pd.read_csv("Data/IMDb movies.csv", low_memory=False)

In [4]:
display(data_raw.head(5))
display(data_raw.shape)
display(data_raw.dtypes)
display(data_raw.columns)

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


(85855, 22)

imdb_title_id             object
title                     object
original_title            object
year                      object
date_published            object
genre                     object
duration                   int64
country                   object
language                  object
director                  object
writer                    object
production_company        object
actors                    object
description               object
avg_vote                 float64
votes                      int64
budget                    object
usa_gross_income          object
worlwide_gross_income     object
metascore                float64
reviews_from_users       float64
reviews_from_critics     float64
dtype: object

Index(['imdb_title_id', 'title', 'original_title', 'year', 'date_published',
       'genre', 'duration', 'country', 'language', 'director', 'writer',
       'production_company', 'actors', 'description', 'avg_vote', 'votes',
       'budget', 'usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics'],
      dtype='object')

In [5]:
data_raw.drop(['year', 'date_published','duration', 'director', 'writer',
              'production_company', 'actors','avg_vote', 'votes', 'budget', 'usa_gross_income',
              'worlwide_gross_income', 'metascore', 'imdb_title_id', 'original_title',
              'reviews_from_users', 'reviews_from_critics'], axis=1,inplace=True)

In [6]:
data_raw.genre.value_counts()

Drama                          12543
Comedy                          7693
Comedy, Drama                   4039
Drama, Romance                  3455
Comedy, Romance                 2508
                               ...  
Family, Sci-Fi, Adventure          1
Adventure, Crime, War              1
Mystery, Romance, Musical          1
Adventure, Sci-Fi, Action          1
Animation, Fantasy, Mystery        1
Name: genre, Length: 1257, dtype: int64

In [7]:
data_raw.head(1)

,title,genre,country,language,description
0,Miss Jerry,Romance,USA,None,The adventures of a female reporter in the 1890s.


In [8]:
countries = data_raw['country'].value_counts()

In [9]:
top_10_countries = countries[:10]
display(top_10_countries)
other_countries = countries[:20:-1]
# display(other_countries) 
#para dropear en caso de decidir no quedarnos con estos datos

USA          28511
India         6065
UK            4111
Japan         3077
France        3055
Italy         2444
Canada        1802
Germany       1396
Turkey        1351
Hong Kong     1239
Name: country, dtype: int64

In [10]:
languages = data_raw['language'].value_counts()
top_10_languages = languages[:10]

In [11]:
missing_values_check = data_raw.isnull().sum()
print(missing_values_check)

title             0
genre             0
country          64
language        833
description    2115
dtype: int64


In [12]:
data_raw.dropna(inplace=True)

In [13]:
missing_values_check = data_raw.isnull().sum()
print(missing_values_check)

title          0
genre          0
country        0
language       0
description    0
dtype: int64


In [14]:
## En el caso de que quiera 3 paises.
# mask = (data_raw['country']=='USA') | (data_raw['country']=='Canada') | (data_raw['country']=='UK')
# mask
# data = data_raw.loc[mask, :]

data = data_raw

data.description = data.title + " " + data.description
data.head()

,title,genre,country,language,description
0,Miss Jerry,Romance,USA,None,Miss Jerry The adventures of a female reporter...
1,The Story of the Kelly Gang,"Biography, Crime, Drama",Australia,None,The Story of the Kelly Gang True story of noto...
3,Cleopatra,"Drama, History",USA,English,Cleopatra The fabled queen of Egypt's affair w...
4,L'Inferno,"Adventure, Drama, Fantasy",Italy,Italian,L'Inferno Loosely adapted from Dante's Divine ...
5,"From the Manger to the Cross; or, Jesus of Naz...","Biography, Drama",USA,English,"From the Manger to the Cross; or, Jesus of Naz..."


In [15]:
data.shape

(82887, 5)

In [16]:
import unidecode
import re

def clean_text(t):
    t_lower_no_accents=unidecode.unidecode(t.lower()); # sacamos acentos y llevamos a minuscula
    t_lower_no_accents_no_punkt=re.sub(r'([^\s\w]|_)+','',t_lower_no_accents); # quitamos signos de puntuacion usando una regex que reemplaza todo lo q no sean espacios o palabras por un string vacio
    return t_lower_no_accents_no_punkt

In [17]:
data['description_clean'] = data['description'].apply(clean_text)

In [18]:
data.head()

,title,genre,country,language,description,description_clean
0,Miss Jerry,Romance,USA,None,Miss Jerry The adventures of a female reporter...,miss jerry the adventures of a female reporter...
1,The Story of the Kelly Gang,"Biography, Crime, Drama",Australia,None,The Story of the Kelly Gang True story of noto...,the story of the kelly gang true story of noto...
3,Cleopatra,"Drama, History",USA,English,Cleopatra The fabled queen of Egypt's affair w...,cleopatra the fabled queen of egypts affair wi...
4,L'Inferno,"Adventure, Drama, Fantasy",Italy,Italian,L'Inferno Loosely adapted from Dante's Divine ...,linferno loosely adapted from dantes divine co...
5,"From the Manger to the Cross; or, Jesus of Naz...","Biography, Drama",USA,English,"From the Manger to the Cross; or, Jesus of Naz...",from the manger to the cross or jesus of nazar...


In [19]:
data['genre_clean'] = data['genre'].apply(clean_text)
# data['genre_list'] = data['genre_clean'].str.split(" ")

In [20]:
genres = pd.unique(data['genre_clean'].str.split(expand=True).stack())
genres

array(['romance', 'biography', 'crime', 'drama', 'history', 'adventure',
       'fantasy', 'war', 'mystery', 'horror', 'western', 'comedy',
       'family', 'action', 'scifi', 'thriller', 'sport', 'animation',
       'musical', 'music', 'filmnoir', 'adult', 'documentary',
       'realitytv', 'news'], dtype=object)

In [21]:
data.head(5)

,title,genre,country,language,description,description_clean,genre_clean
0,Miss Jerry,Romance,USA,None,Miss Jerry The adventures of a female reporter...,miss jerry the adventures of a female reporter...,romance
1,The Story of the Kelly Gang,"Biography, Crime, Drama",Australia,None,The Story of the Kelly Gang True story of noto...,the story of the kelly gang true story of noto...,biography crime drama
3,Cleopatra,"Drama, History",USA,English,Cleopatra The fabled queen of Egypt's affair w...,cleopatra the fabled queen of egypts affair wi...,drama history
4,L'Inferno,"Adventure, Drama, Fantasy",Italy,Italian,L'Inferno Loosely adapted from Dante's Divine ...,linferno loosely adapted from dantes divine co...,adventure drama fantasy
5,"From the Manger to the Cross; or, Jesus of Naz...","Biography, Drama",USA,English,"From the Manger to the Cross; or, Jesus of Naz...",from the manger to the cross or jesus of nazar...,biography drama


In [22]:
data.shape

(82887, 7)

In [23]:
data.reset_index(inplace=True, drop=True)

In [24]:
data.head(10)

,title,genre,country,language,description,description_clean,genre_clean
0,Miss Jerry,Romance,USA,None,Miss Jerry The adventures of a female reporter...,miss jerry the adventures of a female reporter...,romance
1,The Story of the Kelly Gang,"Biography, Crime, Drama",Australia,None,The Story of the Kelly Gang True story of noto...,the story of the kelly gang true story of noto...,biography crime drama
2,Cleopatra,"Drama, History",USA,English,Cleopatra The fabled queen of Egypt's affair w...,cleopatra the fabled queen of egypts affair wi...,drama history
3,L'Inferno,"Adventure, Drama, Fantasy",Italy,Italian,L'Inferno Loosely adapted from Dante's Divine ...,linferno loosely adapted from dantes divine co...,adventure drama fantasy
4,"From the Manger to the Cross; or, Jesus of Naz...","Biography, Drama",USA,English,"From the Manger to the Cross; or, Jesus of Naz...",from the manger to the cross or jesus of nazar...,biography drama
5,Madame DuBarry,"Biography, Drama, Romance",Germany,German,"Madame DuBarry The story of Madame DuBarry, th...",madame dubarry the story of madame dubarry the...,biography drama romance
6,Quo Vadis?,"Drama, History",Italy,Italian,"Quo Vadis? An epic Italian film ""Quo Vadis"" in...",quo vadis an epic italian film quo vadis influ...,drama history
7,Richard III,Drama,"France, USA",English,Richard III Richard of Gloucester uses manipul...,richard iii richard of gloucester uses manipul...,drama
8,Atlantis,Drama,Denmark,Danish,Atlantis After Dr. Friedrich's wife becomes me...,atlantis after dr friedrichs wife becomes ment...,drama
9,Fantômas - À l'ombre de la guillotine,"Crime, Drama",France,French,Fantômas - À l'ombre de la guillotine Inspecto...,fantomas a lombre de la guillotine inspector ...,crime drama


In [25]:
columns_to_drop = ['documentary', 'realitytv', 'news', 'adult' ]

# columns_to_drop = ['documentary', 'realitytv', 'news', 
#                    'family', 'western', 'musical', 
#                    'music', 'biography', 'war', 
#                    'filmnoir', 'history', 'sport', 
#                    'animation' ]
indexes_to_drop = []

for genre in columns_to_drop:
    genre = data['genre_clean'].apply(lambda x: genre in str(x).lower());
    movies_index = list(genre[genre].index)
    for index in movies_index:
        indexes_to_drop.append(index)

In [26]:
data.shape

(82887, 7)

In [27]:
#Drop de filas donde hay movies de las 4 ultimas categorias
data.drop(index=indexes_to_drop, inplace=True)

In [28]:
data.shape

(82880, 7)

In [29]:
data.columns

Index(['title', 'genre', 'country', 'language', 'description',
       'description_clean', 'genre_clean'],
      dtype='object')

In [30]:
data_final = data.drop(['title', 'genre', 'country', 'language', 'description'], axis=1)
data_final.sample(3)

,description_clean,genre_clean
43299,parole damore a wife and mother begins a downw...,drama family
50132,8 paivaa ensiiltaan vilma is a female prompter...,comedy romance
63887,big bad wolves a series of brutal murders puts...,drama horror thriller


In [31]:
data_final.to_csv(path_or_buf='data/movies_preprocesadas.csv', sep=',',
                   header=True, encoding='utf8', index=False)